## Surfaces of revolution

A 2D surface $S$ is defined by a parametric equation of its $X$ and $Y$ components:

$$
S: t \longrightarrow ( X(t) ; Y(t) )
$$

The corresponding 3D surface of revolution $G$, that is obtained by rotating it around the X axis, is parametrized by:

$$
G: t, \alpha \longrightarrow ( X(t) ; Y(t) \cos \alpha ; Y(t) \sin \alpha )
$$

Where $\alpha$ is the rotation angle.

The derivatives of the 3D surface, given the known derivatives of $S$: $X'$ and $Y'$ are:

$$
\frac{dG}{dt} = ( X'(t) ; Y'(t) \cos \alpha ; Y' (t) \sin \alpha )
$$
$$
\frac{dG}{d\alpha} = ( 0 ; - Y(t) \sin \alpha ; Y(t) \cos \alpha )
$$

3D rays are defined with $P$, the origin point, $v$ a vector and the ray parametric equation $P + uv$ of parameter $u$.

Finding a ray surface intersection is solving the parametric equation:

$$
Q(t, \alpha, u) = G(t, \alpha) - P - uv = 0
$$

The Jacobian of Q is:

$$
J(t, \alpha, u) =
\begin{bmatrix}
X'(t) & 0 & -V_x \\
Y'(t)\cos\alpha & -Y(t)\sin\alpha & -V_y \\
Y'(t)\sin\alpha & Y(t)\cos\alpha & -V_z
\end{bmatrix}
$$

Note that interestingly $J$ doesn't depend on $u$ or $X(t)$.

With a current parameter guess $\theta_n = (t, \alpha, u)$, use Newton method's and solve for $\delta$ in this linear system:

$$
J(\theta_n) \delta = Q(\theta_n)
$$

and update with

$$
\theta_{n+1} = \theta_n + \delta
$$

## Singularity issue

Big problem with the above? The points t=0 create a singularity where alpha is sort of many valued. That might be a problem for derivatives and solving JX=Q.

Also J is not invertible at 0, 0 which is a problem.

Need great numerical stability when Y(t) near zero, because that's the optical axis.

In [ ]:
import torchlensmaker as tlm
import torch
import torch.nn

import pprint

# implement intersect_newton_3D
# draw surfaces and collisions

def surface_to_json(surface):
    nsamples = 10
    tspace = torch.linspace(0, surface.domain()[1], nsamples)
    return {"position": surface.pos.tolist(),
    "samples": surface.evaluate(tspace).tolist()}


def rays_to_json(rays, length):
    rays_start = rays[:, :3]
    rays_end = rays_start + length*rays[:, 3:]
    return torch.hstack((rays_start, rays_end)).tolist()


test_shapes = [
    #tlm.BezierSpline(15.0, X=[3.0], CX=[5.8], CY=[0.2*15/2., 1.2*15./2]),
    #tlm.Line(15.0),
    #tlm.PiecewiseLine(height=18.0, X=[-1.0, -3.0, -3.5, -7.0]),
    tlm.CircularArc(20.0, -25.),
    tlm.CircularArc(20.0, 25.),
    tlm.Parabola(20.0, a=0.02),
    tlm.Parabola(35.0, a=0.05),
]

test_surfaces = [
    tlm.Surface(s, torch.tensor([i*5, 0])) for i, s in enumerate(test_shapes)
]


def make_random_rays(num_rays, start_x, end_x, max_y):
    rays_start = (torch.rand((num_rays, 3))*2 - 1) * max_y
    rays_start[:, 0] = start_x

    rays_end = (torch.rand((num_rays, 3))*2 - 1) * max_y
    rays_end[:, 0] = end_x

    rays_vectors = torch.nn.functional.normalize(rays_end - rays_start, dim=1)

    return torch.hstack((rays_start, rays_vectors))


test_rays = make_random_rays(
    num_rays=5,
    start_x=-8,
    end_x=50,
    max_y=6,
)

# debug newton 3D:
# iteration plot of theta (t, alpha, u)
# plot of collision point in space

def newton_step_3d(surface, rays, theta):
    t, alpha, u = theta[:, 0], theta[:, 1], theta[:, 2]

    P, V = rays[:, :3], rays[:, 3:]
    S = surface.evaluate(t)
    Sp = surface.derivative(t)

    print("S", S)
    print("Sp", Sp)

    # Jacobian
    J = torch.stack([
            torch.stack([Sp[:, 0], Sp[:, 1] * torch.cos(alpha), Sp[:, 1] * torch.sin(alpha)], dim=1),
            torch.stack([torch.zeros_like(t), -S[:, 1] * torch.sin(alpha), S[:, 1] * torch.cos(alpha)], dim=1),
            -V
        ], dim=1)

    # Q
    G = torch.stack((S[:, 0], S[:, 1] * torch.cos(alpha), S[:, 1] * torch.sin(alpha)), dim=-1)

    Q = G - P - u.unsqueeze(1).expand_as(V)*V

    # Equation to solve is JX = Q

    ## This fails with
    ## _LinAlgError: torch.linalg.solve: (Batch element 0): The solver failed because the input matrix is singular.
    ## nice try :(
    
    return torch.linalg.solve(J, Q)


def intersect_newton_3d(surface, rays):
    """
    surface ray collision detection in 3D using Newton's method

    rays: tensor of shape (N, 6) where N is the batch dimension and columns are:
          X, Y, Z, VX, VY, VZ
    """

    assert isinstance(rays, torch.Tensor) and rays.dim() == 2
    assert rays.shape[1] == 6

    # Initialize solutions: theta = (t, alpha, u)
    theta = torch.zeros(rays.shape[0], 3)

    with torch.no_grad():
        for _ in range(20):  # TODO parameters for newton iterations
            theta = newton_step_3d(surface, rays, theta)
            print(theta)
            print()

    # One newton iteration for backwards pass
    theta = newton_step_3d(surface, rays, theta)

    return theta

def render():
    data = {}

    data["surfaces"] = [
        surface_to_json(s) for s in test_surfaces]

    data["rays"] = rays_to_json(test_rays, 100)
    
    #pprint.pprint(data["rays"])
    tlm.viewer(data)
    

def demo(rays):

    #intersect_newton_3d(test_surfaces[0], rays)
    
    render()
    


demo(test_rays)